In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
from pyspark.sql import functions as F  #filtering
import geopandas as gpd
import folium
import pandas as pd
import numpy as np

In [2]:
# starting a Spark session
spark = (
    SparkSession.builder.appName('Domain Data Inspection')
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "2g")
    .config("spark.executor.memory", "4g")
    .getOrCreate()
)

# read the parquet dataset
domain = spark.read.parquet('../data/curated/domain_data')
domain.limit(5)

your 131072x1 screen size is bogus. expect trouble
24/09/23 20:10:08 WARN Utils: Your hostname, LAPTOP-KOQUIUN resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/09/23 20:10:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/23 20:10:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/09/23 20:10:09 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/09/23 20:10:15 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


url,price,address,property_type,latitude,longitude,Beds,Baths,Parking,bond,extracted_price,geometry,sa2_code,sa2_name,chg_flag,chg_lbl,sa3_code,sa3_name,sa4_code,sa4_name,gcc_code,gcc_name,ste_code,ste_name,aus_code,aus_name,areasqkm,loci_uri,__index_level_0__
https://www.domai...,"$1,400.00","10 Allara Court, ...",Townhouse,-37.77427300000001,145.1811258,4.0,3.0,2.0,9125.0,1400.0,[01 01 00 00 00 C...,211021261,Donvale - Park Or...,0,No change,21102,Manningham - East,211,Melbourne - Outer...,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,20.8028,http://linked.dat...,0
https://www.domai...,$750 per week,"7 Pine Ridge, Don...",House,-37.7912513,145.1756489,4.0,2.0,0.0,3259.0,750.0,[01 01 00 00 00 8...,211021261,Donvale - Park Or...,0,No change,21102,Manningham - East,211,Melbourne - Outer...,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,20.8028,http://linked.dat...,1
https://www.domai...,$1300 per week,"20 Mulsanne Way, ...",House,-37.7972323,145.1812636,5.0,2.0,2.0,5649.0,1300.0,[01 01 00 00 00 9...,211021261,Donvale - Park Or...,0,No change,21102,Manningham - East,211,Melbourne - Outer...,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,20.8028,http://linked.dat...,2
https://www.domai...,$825pw / $3585pcm,3 Monterey Cresce...,House,-37.792402,145.1743233,3.0,1.0,5.0,3585.0,825.0,[01 01 00 00 00 C...,211021261,Donvale - Park Or...,0,No change,21102,Manningham - East,211,Melbourne - Outer...,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,20.8028,http://linked.dat...,3
https://www.domai...,$680.00,3/49 Leslie Stree...,Townhouse,-37.7810117,145.180705,3.0,2.0,2.0,2955.0,680.0,[01 01 00 00 00 2...,211021261,Donvale - Park Or...,0,No change,21102,Manningham - East,211,Melbourne - Outer...,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,20.8028,http://linked.dat...,4


In [3]:
from pyspark.sql import functions as F

# Check for null values in each column
domain.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in domain.columns]).show()

+---+-----+-------+-------------+--------+---------+----+-----+-------+----+---------------+--------+--------+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+-----------------+
|url|price|address|property_type|latitude|longitude|Beds|Baths|Parking|bond|extracted_price|geometry|sa2_code|sa2_name|chg_flag|chg_lbl|sa3_code|sa3_name|sa4_code|sa4_name|gcc_code|gcc_name|ste_code|ste_name|aus_code|aus_name|areasqkm|loci_uri|__index_level_0__|
+---+-----+-------+-------------+--------+---------+----+-----+-------+----+---------------+--------+--------+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+-----------------+
|  0|    0|      0|            0|       0|        0|   0|    0|      0|1199|              0|       0|       0|       0|       0|      0|       0|       0|       0|       0|       0|       0|       0|       0|   

In [4]:
# Get number of rows and columns in a Spark DataFrame
row_count = domain.count()  # Get the number of rows
column_count = len(domain.columns)  # Get the number of columns

# Display the number of rows and columns
print(f"Rows: {row_count}, Columns: {column_count}")

Rows: 9565, Columns: 29
